In [ ]:
import dask.distributed as dd
from dask.distributed import Client, LocalCluster, progress
from dask_jobqueue import PBSCluster
from dask.distributed import get_worker
import os

In [ ]:
import os
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory to the workshop directory
# which was created in your username directory under /scratch/vp91

os.chdir(os.path.expandvars("/scratch/vp91/$USER/"))

In [ ]:
# Make sure the python we use is from the venv

os.environ['DASK_PYTHON'] = '/g/data/vp91/Training-Venvs/intro-to-dask/bin/python3'

In [ ]:
# Make sure all the modules are loaded.
# It is essential that we use the same python and library for all aspects of dask
# If we dont activate the venv then the workers may have a different versions of libraries

setup_commands = ["module load python3/3.11.0", 
                  "source /g/data/vp91/Training-Venvs/intro-to-dask/bin/activate"]

In [ ]:
# Gadi use custom PBS directives
# So some of the default values to launch a PBS job through Dask call will not work in Gadi
# Any directive specific to gadi should be mentioned here.
# refer : https://opus.nci.org.au/display/Help/Gadi+Quick+Reference+Guide
extra = ['-q normal',
         '-P vp91', 
         '-l ncpus=48', 
         '-l mem=192GB',
         '-l storage=scratch/vp91+gdata/vp91']

In [ ]:
# walltime: Walltime for each worker job.
# cores: Total number of cores per job.
# shebang: Path to desired interpreter for your batch submission script.
# job_extra_directives: List of other PBS options. Each option will be prepended with the #PBS prefix.
# local_directory: Dask worker local directory for file spilling.
# job_directives_skip: Directives to skip in the generated job script header. Directives lines containing 
#                      the specified strings will be removed. Directives added by job_extra_directives 
#                      won’t be affected.
# interface: Network interface like ‘eth0’ or ‘ib0’. This will be used both for the Dask scheduler and 
#            the Dask workers interface
# job_script_prologue: Commands to add to script before launching worker
# python: Python executable used to launch Dask workers. Defaults to the Python that is submitting these jobs



cluster = PBSCluster(walltime="00:20:00", 
                     cores=48, 
                     memory="192GB",
                     shebang='#!/usr/bin/env bash',
                     job_extra_directives=extra, 
                     local_directory='$TMPDIR', 
                     job_directives_skip=["select"], 
                     interface="ib0",
                     job_script_prologue=setup_commands,
                     python=os.environ["DASK_PYTHON"])

In [ ]:
print(cluster.job_script())

In [ ]:
# create a cluster with 2 nodes
cluster.scale(jobs=2)

In [ ]:
# Verify the workers have been allocated as expected
!qstat

In [ ]:
cluster

In [ ]:
# create the client
client = Client(cluster)

In [ ]:
client

cluster.get_logs()

In [ ]:
# A simple test function
def slow_increment(x):
    return x+1

In [ ]:
# Submit the work to the Dask cluster
futures = client.submit(slow_increment, 5000)

In [ ]:
futures

In [ ]:
futures.result()

In [ ]:
import dask.dataframe as df

ddf = df.read_csv(
    os.path.join("intro-to-dask/data", "nycflights", "*.csv"),
    dtype={"TailNum": str, "CRSElapsedTime": float, "Cancelled": bool},
)

In [ ]:
len(ddf)

In [ ]:
import dask.array as da

# Create two large random arrays
a = da.random.random((20000, 20000), chunks=(2000, 2000))
b = da.random.random((20000, 20000), chunks=(2000, 2000))

# Matrix multiplication (this creates a large graph of tasks)
c = da.dot(a, b)

# Trigger computation
result = c.mean().compute()

print("Mean of result:", result)
